<a name="top" id="top"></a>

<div align="center">
    <h1>Representación y Simulación de Circuitos Cuánticos con Qudits en Cirq</h1>
    <a href="https://github.com/qcrbellor">Cristian E. Bello</a>
    <br>
    <i>Departamento de Matemáticas, Universidad Nacional de Colombia</i>
    <br>
    <i>Medellín, Antioquia, 050034, Colombia</i>
    <br>
    <br>
    <a href="https://colab.research.google.com/github/QuantumColombiaUNAL/Curso_Qudits/blob/main/Sesion3_Agosto15.ipynb" target="_parent">
        <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
    </a>
</div>

<a id="1"></a>
# 1. Introducción


<a id="1.1"></a>
## 1.1. Repositorio

Los notebooks para las sesiones prácticas y teóricas se encuentran disponibles en el repositorio: [https://github.com/QuantumColombiaUNAL/Curso_Qudits](https://github.com/QuantumColombiaUNAL/Curso_Qudits)

<a id="2"></a>
## 2. Framework: Cirq

Creado por **Google Quantum AI**, Cirq es un framework de Python para crear, editar y ejecutar circuitos cuánticos en simuladores cuánticos y hardware cuántico.

- Página principal: https://quantumai.google/cirq
- Tutoriales: https://quantumai.google/cirq/start/start
- API: https://quantumai.google/reference/python/cirq/all_symbols
- Repositorio: https://github.com/quantumlib/Cirq
- Instalación:
```
    pip install cirq
```

<a id="3"></a>
## 3. Plataforma: Google Quantum AI

Google Quantum AI trabaja en el desarrollo de procesadores cuánticos y algoritmos cuánticos. Aunque su acceso es más restringido comparado con otras plataformas, Google ha hecho avances significativos en la computación cuántica, como demostrar la ventaja cuántica. El acceso a las computadoras cuánticas de Google ha sido principalmente a través de colaboraciones académicas y de investigación, a diferencia de otras plataformas, Google no ha ofrecido un acceso público generalizado a sus computadoras cuánticas a través de la nube para individuos o empresas.

Utiliza procesadores cuánticos superconductores.

- Página principal: https://quantumai.google/

<a id="4"></a>
# 4. Instalación

Usaremos **Cirq** en esta sesión práctica, debido a que es uno de los frameworks más populares para programación con qudits en computación cuántica.

<a id="4.1"></a>
## 4.1. Ejecución remota

Es posible ejecutar los notebooks desde ambientes remotos como [Google Colab](https://colab.research.google.com/). Es necesario contar con una cuenta de [Gmail](https://mail.google.com/mail/u/0/) (o compatible).

Se sube cada notebook por separado y cuando la sesión sea reiniciada, se debe instalar Cirq:

In [14]:
!pip install cirq

<a id="5"></a>
# 5. Qudits

Ya hemos trabajado con qubits, sistemas cuánticos de $2$ niveles. Veremos como es posible definir sistemas de mayor dimensión. Un ***qudit*** es la generalización de un qubit a un sistema de ***d*** niveles o ***d*** dimensiones.

**Ejemplo:**

1.   Superposición de dos estados base para un qubit. $|\psi\rangle=\alpha|0\rangle+\beta|1\rangle$
2.   Superposición de tres estados base para un ***qudit***: $|\psi\rangle=\alpha|0\rangle+\beta|1\rangle+\gamma|2\rangle$


Un **qubit** tiene dimensión $2$, un **qutrit** tiene dimensión $3$, un **ququart** tiene dimensión $4$, y así sucesivamente.

En Cirq, los qudits funcionan exactamente igual que los qubits, salvo que tienen un atributo `dimension` distinto de $2$, y sólo pueden utilizarse con puertas específicas de esa dimensión. En cirq, tanto los qubits como los qudits son subclases de la clase `cirq.Qid`. Para aplicar una compuerta a qudits, las dimensiones de los qudits deben coincidir con las dimensiones sobre las que trabaja.

**Ejemplo:** consideremos la puerta que representa una evolución unitaria sobre tres qudits, además que hay un qubit, y dos qutrits. Entonces la «qid shape» de la puerta es `(2, 3, 3)` y el método `on` acepta exactamente $3$ `Qid` con dimensión $2$, $3$, y $3$, respectivamente.

In [15]:
import cirq
import numpy as np

class QutritPlusGate(cirq.Gate):
    """Una puerta que suma uno a la base incial de un qutrit.

    Esta puerta actúa en sistemas de tres niveles. En la base computacional de
    este sistema se ejecuta la transformación U|x〉 = |x + 1 mod 3〉, es
    decir, U|0〉 = |1〉, U|1〉 = |2〉, y U|2> = |0〉.
    """

    def _qid_shape_(self):
        # Al aplicar este método, se implementa el protocolo
        # cirq.qid_shape y devuelve la tupla (3,) cuando cirq.qid_shape actúa
        # sobre una instancia de la clase. La puerta actúa sobre un único qutrit.
        return (3,)

    def _unitary_(self):
        # Como la puerta actúa sobre sistemas de tres niveles tiene un efecto unitario
        # En efecto, es una matriz unitaria tres por tres.
        return np.array([[0, 0, 1],
                         [1, 0, 0],
                         [0, 1, 0]])

    def _circuit_diagram_info_(self, args):
        return '[+1]'

# Creamos un qutrit.
q0 = cirq.LineQid(0, dimension=3)

# Aplicamos la puerta a este qutrit.
circuit = cirq.Circuit(
    QutritPlusGate().on(q0)
)

# Qutrit etiquetado con su dimensión.
print(circuit)

0 (d=3): ───[+1]───


<a id="6"></a>
# 6. Representación

## 6.1. cirq.Qid

`cirq.Qid` es la clase que representa tanto qubits como qudits en Cirq.

Cirq incorpora las clases de qubit, `cirq.NamedQubit`, `cirq.GridQubit`, y `cirq.LineQubit`, también proporciona los tipos `cirq.Qid`:

- `cirq.NamedQid`
  - **Ejemplo:** Crea un qutrit llamado '**a**' especificando la dimensión: `cirq.NamedQid('a', dimension=3)`.
- `cirq.GridQid`
  - **Ejemplo:** Crea un qutrit en la posición (2, 0) especificando la dimensión: `cirq.GridQid(2, 0, dimension=3)`.
  - **Ejemplo:** Se pueden crear regiones de `cirq.GridQid`. Para crear una cuadrícula $2$ x $2$ de ququarts, se utiliza `cirq.GridQid.rect(2, 2, dimension=4)`.
- `cirq.LineQid`
  - **Ejemplo:** Crea un qutrit en la posición $1$ de la línea especificando la dimensión: `cirq.LineQid(0, dimension=3)`.
  - **Ejemplo:** Se pueden crear rangos de `cirq.LineQid`s. Por ejemplo, para crear qutrits en una línea con ubicaciones de $0$ a $4$, se utiliza `cirq.LineQid.range(5, dimension=3)`.
  
Por defecto, las clases `cirq.Qid` en cirq serán qubits a menos que su parámetro `dimension` se especifique en la creación. Así, un `cirq.Qid` como `cirq.NamedQid('a')` es un qubit.

## 6.2. Protocolo: `cirq.qid_shape`

Las puertas cuánticas, operaciones y otros tipos que actúan sobre una secuencia de qudits pueden especificar la dimensión de cada qudit sobre el que actúan implementando el método `_qid_shape_`.  Este método devuelve una tupla de enteros correspondiente a la dimensión requerida de cada qudit sobre el que opera, por ejemplo `(2, 3, 3)` significa un objeto que actúa sobre un qubit, un qutrit y otro qutrit.  Cuando se especifica `_qid_shape_` decimos que el objeto implementa el protocolo `qid_shape`.

Cuando se utilizan `cirq.Qid`s con `cirq.Gate`s, `cirq.Operation`s, y `cirq.Circuit`s, la dimensión de cada qid debe coincidir con la entrada correspondiente en la forma qid. En caso contrario, se produce un error.

Se puede consultar la forma `Qid` de un objeto o de una lista de `Qid` llamando a `cirq.qid_shape`. Por defecto, `cirq.qid_shape` devolverá la forma `Qid` equivalente para qubits si `_qid_shape_` no está definida.  En particular, para una puerta de sólo qubits, la forma `Qid` es una tupla que contiene un $2$ por cada qubit, por ejemplo `(2,) * cirq.num_qubits(puerta)`.

In [16]:
# Crea una instancia de la puerta qutrit.
gate = QutritPlusGate()

# Verifica que actúa sobre un único qutrit.
print(cirq.qid_shape(gate))

(3,)


## 6.3. Estados unitarios, mixtos y canales en qudits

Los métodos `_unitary_`, `_apply_unitary_`, `_mixture_`, y `_kraus_` se pueden utilizar para definir puertas unitarias, mixtas, y los canales se pueden utilizar con qudits.

Dado que el espacio de estados para qudits para $d>2$ viven en espacios dimensionales más grandes, los objetos correspondientes devueltos por estos métodos son de dimensión superior.

In [17]:
# Crea una instancia de la puerta qutrit. Esta puerta implementa _unitary_.
gate = QutritPlusGate()

# Como actúa sobre qutrits, su unitario es una matriz de 3 x 3.
print(cirq.unitary(gate))

[[0 0 1]
 [1 0 0]
 [0 1 0]]


Para una puerta de un solo qubit, su unitario es una matriz de $2$ x $2$, mientras que para una puerta de un qutrit unitario es una matriz de $3$ x $3$.  Una puerta de dos qutrits tendrá un unitario que es una matriz de $9$ x $9$ ($3 * 3 = 9$) y una puerta qubit-quart tendrá un unitario que es una matriz de $8$ x $8$ ($2 * 4 = 8$).

<a id="7"></a>
# 7. Simulación de qudits

Los simuladores de Cirq pueden utilizarse para simular o muestrear circuitos que actúan sobre qudits.

Simuladores como `cirq.Simulator` y `cirq.DensityMatrixSimulator` devolverán resultados de simulación con estados más grandes que el circuito qubit del mismo tamaño cuando se simulan circuitos qudit. El tamaño del estado devuelto viene determinado por el producto de las dimensiones de los qudits simulados. Por ejemplo, la salida del vector de estado de `cirq.Simulator` después de simular un circuito en un qubit, un qutrit, y un qutrit tendrá $2 * 3 * 3 = 18$ elementos. Se puede llamar a `cirq.qid_shape(simulation_result)` para comprobar las dimensiones del qudit.

In [18]:
# Crea un circuito
q0 = cirq.LineQid(0, dimension=3)
circuit = cirq.Circuit(QutritPlusGate()(q0))

# Realiza la simulación de este circuito.
sim = cirq.Simulator()
result = sim.simulate(circuit)

# Verifica que el estado devuelto es el de un qutrit.
print(cirq.qid_shape(result))

(3,)


Se supone que los circuitos en qudits siempre empiezan en el estado de base computacional $|0\rangle$, y se supone que todos los estados de base computacional de un qudit son $|0\rangle$, $|1\rangle$, ..., $|d-1\rangle$.  En consecuencia, se supone que las mediciones de los qudits están en la base de cálculo y para cada qudit devuelven un número entero correspondiente a estos estados de base.  Así, se supone que los resultados de las mediciones de cada qudits van de $0$ a $d-1$.

In [19]:
# Crea un circuito con tres puertas qutrit.
q0, q1 = cirq.LineQid.range(2, dimension=3)
circuit = cirq.Circuit([
    QutritPlusGate()(q0),
    QutritPlusGate()(q1),
    QutritPlusGate()(q1),
    cirq.measure(q0, q1, key="x")
])

# Muestra de este circuito.
result = cirq.sample(circuit, repetitions=3)

# Muestra el resultado
print(result)

x=111, 222


<div align="center">
    <a href="#top">🔝 Volver al inicio 🔝</a>
</div>